# Scatter plot de frequencia X Nota

- Identificar correlacao de frequencia e nota
- Interatividade deve poder separar por depto

In [31]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pprint import pprint

In [2]:
students = pd.read_csv("../data/students.csv", index_col="IdEstudante")
grades = pd.read_csv("../data/final_table.csv")

grades["depto"] = grades["Sigla"].apply(lambda s: s[:3])

joined_df = grades.join(students, on="student_id", how="inner")
joined_df

,Sigla,Nome,Ano/Semestre,AU,TR,AU+TR,EXT,FREQ,NOTA,student_id,depto,Idade,Curso,AnoIngresso,Ponderada,Classificacao,AnoConclusao,AAC,AEX
0,SCC0200,Informação Profissional e Tutoria Acadêmica em...,2023/1,2,0,2,30,100.0,10.0,0,SCC,20,BCC,2023,9.1,16,2027,80,190
1,SCC0221,Introdução à Ciência de Computação I,2023/1,4,1,5,90,97.0,9.0,0,SCC,20,BCC,2023,9.1,16,2027,80,190
2,SCC0222,Laboratório de Introdução à Ciência de Computa...,2023/1,2,2,4,90,100.0,9.8,0,SCC,20,BCC,2023,9.1,16,2027,80,190
3,SMA0300,Geometria Analítica,2023/1,4,0,4,60,100.0,7.5,0,SMA,20,BCC,2023,9.1,16,2027,80,190
4,SMA0353,Cálculo I,2023/1,4,0,4,60,100.0,10.0,0,SMA,20,BCC,2023,9.1,16,2027,80,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,SME0104,Cálculo Numérico,2025/1,4,0,4,0,100.0,9.5,4,SME,20,BCC,2023,8.9,32,2027,1,180
220,SME0121,Processos Estocásticos,2025/1,4,0,4,0,80.0,8.5,4,SME,20,BCC,2023,8.9,32,2027,1,180
221,SME0130,Redes Complexas,2025/1,4,2,6,0,100.0,10.0,4,SME,20,BCC,2023,8.9,32,2027,1,180
222,SSC0130,Engenharia de Software,2025/1,4,1,5,15,100.0,8.4,4,SSC,20,BCC,2023,8.9,32,2027,1,180


In [17]:
fig = px.scatter(joined_df, x="FREQ", y="NOTA", color="depto", hover_data=["Nome", "FREQ", "NOTA"], title="Scatter Plot de Frequencia x Nota")
fig.show()

# Grafico de areas empilhadas de nota por semestre, mostrando a contribuicao de cada departamento para a media

- Para envio considerar apenas o segundo

In [37]:
sorted(joined_df["Ano/Semestre"].unique())

['2020/1',
 '2020/2',
 '2021/1',
 '2021/2',
 '2022/1',
 '2022/2',
 '2023/1',
 '2023/2',
 '2024/1',
 '2024/2',
 '2025/1']

In [52]:
fig = go.Figure()

for student in students.index:
    semesters = [] # Stores the semesters labels
    averages = [] # Stores the array of contributions to average
    student_df = joined_df[joined_df["student_id"] == student]
    for semester in sorted(joined_df["Ano/Semestre"].unique()):
        student_sem_df = student_df[student_df["Ano/Semestre"] == semester]
        semesters.append(semester)
        if student_sem_df.empty:
            averages.append(None)
            continue
        sem_avgs = {}
        semester_total = 0
        for depto in student_df["depto"].unique():
            student_sem_depto_notas = student_sem_df[student_sem_df["depto"] == depto]["NOTA"]
            avg = student_sem_depto_notas.mean()
            count = student_sem_depto_notas.count()
            avg = avg if count != 0 else 0
            sem_avgs[depto] = (avg, count)
            semester_total += count
        sem_avgs["total"] = semester_total
        averages.append(sem_avgs)
    print(semesters)
    pprint(averages)
    for depto in student_df["depto"].unique():
        y = list(map(lambda x: x[depto][0] * x[depto][1] / x["total"] if x is not None else 0, averages))
        print(semesters)
        fig.add_trace(go.Scatter(
            x=semesters, y=y,
            mode="lines+markers",
            stackgroup="depto",
            name=depto,
        ))
        break

fig.show()


['2020/1', '2020/2', '2021/1', '2021/2', '2022/1', '2022/2', '2023/1', '2023/2', '2024/1', '2024/2', '2025/1']
[None,
 None,
 None,
 None,
 None,
 None,
 {'760': (0, np.int64(0)),
  'SCC': (np.float64(9.6), np.int64(3)),
  'SMA': (np.float64(8.75), np.int64(2)),
  'SME': (0, np.int64(0)),
  'SSC': (np.float64(9.625), np.int64(4)),
  'total': np.int64(9)},
 {'760': (np.float64(9.0), np.int64(1)),
  'SCC': (np.float64(9.733333333333334), np.int64(3)),
  'SMA': (np.float64(9.7), np.int64(2)),
  'SME': (0, np.int64(0)),
  'SSC': (np.float64(9.45), np.int64(2)),
  'total': np.int64(8)},
 {'760': (np.float64(9.2), np.int64(1)),
  'SCC': (np.float64(9.399999999999999), np.int64(2)),
  'SMA': (np.float64(8.9), np.int64(1)),
  'SME': (0, np.int64(0)),
  'SSC': (np.float64(9.333333333333334), np.int64(3)),
  'total': np.int64(7)},
 {'760': (0, np.int64(0)),
  'SCC': (np.float64(8.65), np.int64(2)),
  'SMA': (0, np.int64(0)),
  'SME': (np.float64(7.949999999999999), np.int64(2)),
  'SSC': (np.flo

In [64]:
import plotly.graph_objects as go

fig = go.Figure()


all_student_data = []
trace_count_per_student = []

for student in students.index:
    semesters = []  # Stores the semesters labels
    averages = []  # Stores the array of contributions to average
    student_df = joined_df[joined_df["student_id"] == student]

    for semester in sorted(joined_df["Ano/Semestre"].unique()):
        student_sem_df = student_df[student_df["Ano/Semestre"] == semester]
        semesters.append(semester)
        if student_sem_df.empty:
            averages.append(None)
            continue
        sem_avgs = {}
        semester_total = 0
        for depto in student_df["depto"].unique():
            student_sem_depto_notas = student_sem_df[student_sem_df["depto"] == depto]["NOTA"]
            avg = student_sem_depto_notas.mean()
            count = student_sem_depto_notas.count()
            avg = avg if count != 0 else 0
            sem_avgs[depto] = (avg, count)
            semester_total += count
        sem_avgs["total"] = semester_total
        averages.append(sem_avgs)

    # Students plot
    student_traces = []
    total_y = np.array([0] * len(averages), dtype=np.float64)
    for depto in student_df["depto"].unique():
        y = list(map(lambda x: x[depto][0] * x[depto][1] / x["total"] if x is not None else 0, averages))
        total_y += np.array(y)

        trace = go.Scatter(
            x=semesters,
            y=y,
            mode="lines+markers",
            stackgroup="depto",
            name=depto,
            visible=(student == students.index[0])  # Only first visible
        )
        fig.add_trace(trace)
        student_traces.append(trace)
    fig.add_trace(go.Scatter(
        x=semesters,
        y=total_y,
        mode="lines",
        name="Média do semestre",
        line=dict(width=0),
        showlegend=False,
        hovertemplate="<b>Média total: %{y:.2f}</b><extra></extra>",
        visible=(student == students.index[0])
    ))

    trace_count_per_student.append(len(student_df["depto"].unique())+1)

# Create dropdown buttons
buttons = []
for i, student in enumerate(students.index):
    # Calculate which traces belong to this student
    start_idx = sum(trace_count_per_student[:i])
    end_idx = start_idx + trace_count_per_student[i]

    # Create visibility list
    visible = [False] * len(fig.data)
    for j in range(start_idx, end_idx):
        visible[j] = True

    buttons.append(
        dict(
            label=f"Estudante {student}",
            method="update",
            args=[{"visible": visible},
                  {"title": f"Contribuição à Média por departamento para Estudante {student}"}]
        )
    )

# Add dropdown menu to layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=False,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top",
            bgcolor="gray",
            bordercolor="gray",
            borderwidth=1
        )
    ],
    title=f"Contribuição à Média por departamento para Estudante {students.index[0]}",
    xaxis_title="Semestre",
    yaxis_title="Média",
    hovermode='x unified'
)

fig.show()